In [1]:
import numpy as np
import pylidc as pl
from utils import normalize, get_pixels_from_hu, get_intersection, get_union
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score 
from itertools import combinations 


In [2]:
scans = pl.query(pl.Scan)

In [3]:
def get_average_interrater_agreement(masks):
    n_raters = len(masks)
    rater_combinations = combinations(range(n_raters), 2)
    ira = []
    for rater1, rater2 in list(rater_combinations):
        ira.append(cohen_kappa_score(masks[rater1].ravel(), masks[rater2].ravel()))
        
    average_ira = np.mean(ira) * 100
    print("Average IRA for {} annotators: {}".format(n_raters, average_ira))
    
    

In [4]:
for scan in scans:
    print("\n")
    scan.verbose = False
    
    print("Patient: ", scan.patient_id)
    
    # Get scan volume from DICOM Images
    try:
        np_vol = get_pixels_from_hu(scan.load_all_dicom_images(verbose=False))
    except:
        # Skip scan since we only look at selected subset of LIDC
        print("Skipped Scan!")
        continue
        
    # Create empty volumes to hold annotations from all 4 annotators
    mask_ann = [np.zeros(np_vol.shape, dtype=np.bool).T for i in range(4)]
    
    # Cluster object from pylidc to obtain nodules
    cluster = scan.cluster_annotations()

    # For each nodule in the scan
    for nodule in cluster:
        if len(nodule) > 4:
            continue
            
        # Get each annotation one by one for each nodule
        for ann_idx, ann in enumerate(nodule):
            # Add mask for each annotator to the corresponding mask volume
            mask = ann.boolean_mask()
            bbox = ann.bbox()
            mask_ann[ann_idx][bbox][mask==True] = 1

    # Transpose x-y since viewpoint is rotated
    for idx, mask_vol in enumerate(mask_ann):
        mask_ann[idx] = mask_vol.transpose(2, 0, 1)
    
    get_average_interrater_agreement(mask_ann)
        
        
    
    
    



Patient:  LIDC-IDRI-0078
Average IRA for 4 annotators: 84.65948043425861


Patient:  LIDC-IDRI-0069
Average IRA for 4 annotators: 73.23355338839652


Patient:  LIDC-IDRI-0079


MemoryError: 